# 09-14 실전! 한국어 위키피디아로 Word2Vec 학습하기
---

아래의 실습은 구글의 Colab을 사용한다고 가정하고 작성되었습니다. 다른 환경에서 실습하셔도 상관은 없지만, 형태소 분석기 Mecab은 다른 방법으로 설치하셔야 합니다.

---
## 1. 위키피디아로부터 데이터 다운로드 및 통합
---
위키피디아로부터 데이터를 파싱하기 위한 파이썬 패키지인 wikiextractor를 설치합니다.


In [1]:
!pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 1.8 MB/s eta 0:00:00


위키피디아 데이터를 다운로드 한 후에 전처리에서 사용할 형태소 분석기인 Mecab을 설치합니다.

In [2]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 12.39 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 40.2 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-26 03:57:32--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.2, 18.205.93.1, ...
Connecting to 

위키피디아 덤프(위키피디아 데이터)를 다운로드합니다.

In [3]:
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

--2023-01-26 04:01:02--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 882338866 (841M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2’

kowiki-latest-pages 100%[===================>] 841.46M  5.04MB/s    in 2m 50s  

2023-01-26 04:03:52 (4.94 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2’ saved [882338866/882338866]



위키익스트랙터를 사용하여 위키피디아 덤프를 파싱합니다.

In [4]:
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'kowiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Loaded 61669 templates in 294.5s
INFO: Starting page extraction from kowiki-latest-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (778.9 art/s)
INFO: Extracted 200000 articles (1118.1 art/s)
INFO: Extracted 300000 articles (1255.4 art/s)
INFO: Extrac

현재 경로에 있는 디렉토리와 파일들의 리스트를 받아옵니다.

In [ ]:
!ls

text라는 디렉토리 안에는 또 어떤 디렉토리들이 있는지 파이썬을 사용하여 확인해봅시다.

In [ ]:
import os
import re

In [ ]:
os.listdir('text')

AA라는 디렉토리의 파일들을 확인해봅시다.

In [ ]:
%ls text/AA

텍스트 파일로 변환된 위키피디아 한국어 덤프는 총 6개의 디렉토리로 구성되어져 있습니다. AA ~ AF의 디렉토리로 각 디렉토리 내에는 'wiki_00 ~ wiki_약 90내외의 숫자'의 파일들이 들어있습니다. 다시 말해 각 디렉토리에는 약 90여개의 파일들이 들어있습니다. 각 파일들을 열어보면 이와 같은 구성이 반복되고 있습니다.  
  
```
<doc id="문서 번호" url="실제 위키피디아 문서 주소" title="문서 제목">
내용
</doc>
```

예를 들어서 AA 디렉토리의 wiki_00 파일을 읽어보면, 지미 카터에 대한 내용이 나옵니다.  
  
```
<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령(1977년 ~ 1981년)이다.
지미 카터는 조지아 주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대
위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.
... 이하 중략...
</doc>
```

이제 이 6개 AA ~ AF 디렉토리 안의 wiki 숫자 형태의 수많은 파일들을 하나로 통합하는 과정을 진행해야 합니다. AA ~ AF 디렉토리 안의 모든 파일들의 경로를 파이썬의 리스트 형태로 저장합니다.

In [ ]:
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            # 재귀 함수
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [ ]:
filepaths = list_wiki('text')

총 파일의 개수를 확인해봅시다.

In [ ]:
len(filepaths)

총 파일의 개수는 850개입니다. 이제 output_file.txt라는 파일에 850개의 파일을 전부 하나로 합칩니다.

In [ ]:
with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

파일을 읽고 10줄만 출력해보겠습니다.

In [ ]:
f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

---
##2. 형태소 분석
---

In [ ]:
from tqdm import tqdm
from konlpy.tag import Mecab 

형태소 분석기 Mecab을 사용하여 토큰화를 진행해보겠습니다.

In [ ]:
mecab = Mecab()

우선 output_file에는 총 몇 줄이 있는지 확인합니다.

In [ ]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

9,718,793개의 줄이 존재합니다. 상위 10개만 출력해봅시다.

In [ ]:
lines[:10]

두번째 줄을 보면 아무런 단어도 들어있지 않은 ''와 같은 줄도 존재합니다. 해당 문자열은 형태소 분석에서 제외하도록 하고 형태소 분석을 수행해봅시다.

In [ ]:
result = []

for line in tqdm(lines):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result.append(mecab.morphs(line))

빈 문자열은 제외하고 형태소 분석을 진행했습니다. 이제 몇 개의 줄. 즉, 몇 개의 문장이 존재하는지 확인해봅시다.

In [ ]:
len(result)

6,559,314개로 문장의 수가 줄었습니다.

---
##3. Word2Vec 학습
---
형태소 분석을 통해서 토큰화가 진행된 상태이므로 Word2Vec을 학습합니다.


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

In [ ]:
model_result2 = model.wv.most_similar("어벤져스")
print(model_result2)

In [ ]:
model_result3 = model.wv.most_similar("반도체")
print(model_result3)